In [1]:
## 예시주소 : 부산광역시 서구 서대신동3가 293-6번지
tmp_addr = input("주소를 입력해주세요.\n")

주소를 입력해주세요.
부산광역시 서구 서대신동3가 293-6번지


In [2]:
import folium
import pandas as pd

In [3]:
obs_list = pd.read_csv("관측소좌표.csv", engine='python')

In [4]:
obs_building = pd.read_csv("1to5000데모용.csv", engine='python')

In [5]:
def find_near_site(x, y):
    tmp_near_list = (x - obs_list['위도'][0])**2 + (y - obs_list['경도'][0])**2
    tmp_near_list_index = 0
    for i in range(1, len(obs_list)):
        tmp_near_list_2 = (x - obs_list['위도'][i])**2 + (y - obs_list['경도'][i])**2
        tmp_near_list_index_2 = i
        if tmp_near_list_2 <= tmp_near_list:
            tmp_near_list = tmp_near_list_2
            tmp_near_list_index = tmp_near_list_index_2
    return tmp_near_list, tmp_near_list_index

In [6]:
obs_building["대지_위치"] = obs_building["대지_위치"].astype('str') 
tmp_find_building = obs_building.loc[obs_building["대지_위치"] == tmp_addr]
tmp_find_building_T = tmp_find_building.T.iloc[[0,2,4,5,6,7,24,25,26,27,28,29,30,35,36,40,41,42,43,44,47,48,49]]
tmp_find_building_T.to_csv('tmp_building.csv', encoding='cp949')

In [7]:
tmp_input_x, tmp_input_y = tmp_find_building[["경도","위도"]]["위도"].iloc[0],tmp_find_building[["경도","위도"]]["경도"].iloc[0]     ##임의의 좌표 입력
tmp_def = find_near_site(tmp_input_x, tmp_input_y)     ## 최단거리 찾는 기능 사용

map_osm = folium.Map(location=[tmp_input_x, tmp_input_y], zoom_start=15)
for i in range(0, len(obs_list)):
    folium.Marker([tmp_input_x, tmp_input_y], popup='건물위치',icon=folium.Icon(color='red',icon='info-sign')).add_to(map_osm)
    folium.Marker([obs_list['위도'][float(tmp_def[1])], obs_list['경도'][float(tmp_def[1])]], popup=obs_list['관측소'][tmp_def[1]]).add_to(map_osm)

map_osm               ##전체 맵에 자료 입력하는 예시

In [8]:
## 선택한 위치 인근에 있는 관측소 정보 확인
obs_list.loc[tmp_def[1]]

관측소                    부산동대신
주소          부산 서구 동대신동 2가 98
경도                   129.028
위도                   35.1136
조사년도                    2016
암반층관측정수위              106.67
암반층관측정표고              141.28
암반층관측정심도               34.61
충적층관측정수위                   -
충적층관측정표고                   -
충적층관측정심도                   -
Name: 130, dtype: object

In [9]:
map_osm.save('find_near_point.html')      